# Lambda School Data Science Module 142
## Sampling, Confidence Intervals, and Hypothesis Testing

## Prepare - examine other available hypothesis tests

If you had to pick a single hypothesis test in your toolbox, t-test would probably be the best choice - but the good news is you don't have to pick just one! Here's some of the others to be aware of:

In [1]:
import numpy as np
from scipy.stats import chisquare  # One-way chi square test

# Chi square can take any crosstab/table and test the independence of rows/cols
# The null hypothesis is that the rows/cols are independent -> low chi square
# The alternative is that there is a dependence -> high chi square
# Be aware! Chi square does *not* tell you direction/causation

ind_obs = np.array([[1, 1], [2, 2]]).T
print(ind_obs)
print(chisquare(ind_obs, axis=None))

dep_obs = np.array([[16, 18, 16, 14, 12, 12], [32, 24, 16, 28, 20, 24]]).T
print(dep_obs)
print(chisquare(dep_obs, axis=None))

[[1 2]
 [1 2]]
Power_divergenceResult(statistic=0.6666666666666666, pvalue=0.8810148425137847)
[[16 32]
 [18 24]
 [16 16]
 [14 28]
 [12 20]
 [12 24]]
Power_divergenceResult(statistic=23.31034482758621, pvalue=0.015975692534127565)


In [2]:
# Distribution tests:
# We often assume that something is normal, but it can be important to *check*

# For example, later on with predictive modeling, a typical assumption is that
# residuals (prediction errors) are normal - checking is a good diagnostic

from scipy.stats import normaltest
# Poisson models arrival times and is related to the binomial (coinflip)
sample = np.random.poisson(5, 1000)
print(normaltest(sample))  # Pretty clearly not normal

NormaltestResult(statistic=58.11589277325195, pvalue=2.4004586476814666e-13)


In [0]:
# Kruskal-Wallis H-test - compare the median rank between 2+ groups
# Can be applied to ranking decisions/outcomes/recommendations
# The underlying math comes from chi-square distribution, and is best for n>5
from scipy.stats import kruskal

x1 = [1, 3, 5, 7, 9]
y1 = [2, 4, 6, 8, 10]
print(kruskal(x1, y1))  # x1 is a little better, but not "significantly" so

x2 = [1, 1, 1]
y2 = [2, 2, 2]
z = [2, 2]  # Hey, a third group, and of different size!
print(kruskal(x2, y2, z))  # x clearly dominates

KruskalResult(statistic=0.2727272727272734, pvalue=0.6015081344405895)
KruskalResult(statistic=7.0, pvalue=0.0301973834223185)


And there's many more! `scipy.stats` is fairly comprehensive, though there are even more available if you delve into the extended world of statistics packages. As tests get increasingly obscure and specialized, the importance of knowing them by heart becomes small - but being able to look them up and figure them out when they *are* relevant is still important.

## Live Lecture - let's explore some more of scipy.stats

Candidate topics to explore:

- `scipy.stats.chi2` - the Chi-squared distribution, which we can use to reproduce the Chi-squared test
- Calculate the Chi-Squared test statistic "by hand" (with code), and feed it into `chi2`
- Build a confidence interval with `stats.t.ppf`, the t-distribution percentile point function (the inverse of the CDF) - we can write a function to return a tuple of `(mean, lower bound, upper bound)` that you can then use for the assignment (visualizing confidence intervals)

In [0]:
# Taking requests! Come to lecture with a topic or problem and we'll try it.

## Assignment - Build a confidence interval

A confidence interval refers to a neighborhood around some point estimate, the size of which is determined by the desired p-value. For instance, we might say that 52% of Americans prefer tacos to burritos, with a 95% confidence interval of +/- 5%.

52% (0.52) is the point estimate, and +/- 5% (the interval $[0.47, 0.57]$) is the confidence interval. "95% confidence" means a p-value $\leq 1 - 0.95 = 0.05$.

In this case, the confidence interval includes $0.5$ - which is the natural null hypothesis (that half of Americans prefer tacos and half burritos, thus there is no clear favorite). So in this case, we could use the confidence interval to report that we've failed to reject the null hypothesis.

But providing the full analysis with a confidence interval, including a graphical representation of it, can be a helpful and powerful way to tell your story. Done well, it is also more intuitive to a layperson than simply saying "fail to reject the null hypothesis" - it shows that in fact the data does *not* give a single clear result (the point estimate) but a whole range of possibilities.

How is a confidence interval built, and how should it be interpreted? It does *not* mean that 95% of the data lies in that interval - instead, the frequentist interpretation is "if we were to repeat this experiment 100 times, we would expect the average result to lie in this interval ~95 times."

For a 95% confidence interval and a normal(-ish) distribution, you can simply remember that +/-2 standard deviations contains 95% of the probability mass, and so the 95% confidence interval based on a given sample is centered at the mean (point estimate) and has a range of +/- 2 (or technically 1.96) standard deviations.

Different distributions/assumptions (90% confidence, 99% confidence) will require different math, but the overall process and interpretation (with a frequentist approach) will be the same.

Your assignment - using the data from the prior module ([congressional voting records](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records)):

1. Generate and numerically represent a confidence interval
2. Graphically (with a plot) represent the confidence interval
3. Interpret the confidence interval - what does it tell you about the data and its distribution?

Stretch goals:

1. Write a summary of your findings, mixing prose and math/code/results. *Note* - yes, this is by definition a political topic. It is challenging but important to keep your writing voice *neutral* and stick to the facts of the data. Data science often involves considering controversial issues, so it's important to be sensitive about them (especially if you want to publish).
2. Apply the techniques you learned today to your project data or other data of your choice, and write/discuss your findings here.
3. Refactor your code so it is elegant, readable, and can be easily run for all issues.

In [2]:
import pandas as pd
politics = pd.read_csv('politics.csv')
politics.head()

,party,handicap,water_cost_share,budget_resolution,physician_fee_freeze,el_savador_aid,religious_groups_schools,anti_satellite_test_ban,nicaraguan_aid,mx_missile,immigration,synfuels_cutback,education_spending,superfund_lawsuit,crime,duty_free_exports,s_africa_export
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.5,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.5
2,democrat,0.5,1.0,1.0,0.5,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,democrat,0.0,1.0,1.0,0.0,0.5,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.5,1.0,1.0,1.0,1.0


In [0]:

politics = politics.set_index('party')
repub = politics.loc['republican']
repub.name = 'Republicans'
demo = politics.loc['democrat']
demo.name = 'Democrats'

In [8]:
#find number of obvservations
n = len(demo['handicap'])
print('number of obs:',n)
#find the mean
mean = demo['handicap'].mean()
print('mean:',mean)
#find the standard deviation
std = demo['handicap'].std()
print('standard deviation:',std)
#decide your confidence level: 95% and get z value (why z?)
z_value = 1.960
upper_bound=mean+z_value*std/n**(1/2)
lower_bound=mean-z_value*std/n**(1/2)

print(upper_bound,lower_bound)


number of obs: 267
mean: 0.601123595505618
standard deviation: 0.48188876091874716
0.658926177449035 0.543321013562201


In [15]:
import statsmodels.stats.api as sms
a, b = sms.DescrStatsW(demo['handicap']).tconfint_mean()
a

0.5430578842906294

In [0]:
da=pd.DataFrame()
'''def dew_it(dataframe):
  for column in dataframe:
    l,u = sms.DescrStatsW(column).tconfint_mean()
    df.append(dataframe.name,,l,u)'''
    
def do_it(df,issue):
  l,u = sms.DescrStatsW(df[issue]).tconfint_mean()
  global da
  da = da.append({'party':df.name,'issue':issue,'lower':l,'upper':u},ignore_index=True)

      
    

In [0]:
da=pd.DataFrame()
for column in demo:
  do_it(demo,column)

In [62]:
da

,issue,lower,party,upper
0,handicap,0.543058,Democrats,0.659189
1,water_cost_share,0.444765,Democrats,0.558981
2,budget_resolution,0.840034,Democrats,0.916520
3,physician_fee_freeze,0.039006,Democrats,0.095825
4,el_savador_aid,0.179422,Democrats,0.277507
5,religious_groups_schools,0.418255,Democrats,0.536801
6,anti_satellite_test_ban,0.713863,Democrats,0.814227
7,nicaraguan_aid,0.778591,Democrats,0.869349
8,mx_missile,0.689234,Democrats,0.790167
9,immigration,0.412099,Democrats,0.531721


In [0]:
for column in repub:
  do_it(repub,column)

In [64]:
da

,issue,lower,party,upper
0,handicap,0.543058,Democrats,0.659189
1,water_cost_share,0.444765,Democrats,0.558981
2,budget_resolution,0.840034,Democrats,0.916520
3,physician_fee_freeze,0.039006,Democrats,0.095825
4,el_savador_aid,0.179422,Democrats,0.277507
5,religious_groups_schools,0.418255,Democrats,0.536801
6,anti_satellite_test_ban,0.713863,Democrats,0.814227
7,nicaraguan_aid,0.778591,Democrats,0.869349
8,mx_missile,0.689234,Democrats,0.790167
9,immigration,0.412099,Democrats,0.531721


In [0]:
da.to_csv('da.csv')